In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import re
import json

In [2]:
class Diputados(scrapy.Spider):
    name = "spider_diputados"
    
    def start_requests(self):
        links = [urlCurr + i for i in terminaciones]
        for link in links:
            yield scrapy.Request( url = link, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'}, 
                              callback = self.parse )
    
    def parse(self,response):
        nombre = response.css('td.textocurrienc > center > strong::text').get()
        idDipu = ''.join([c for c in nombre[1:] if c.isupper()])
        cumple = response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[6]
        curp = creaCurp(nombre,cumple)
        dipus[curp]={}
        dipus.get(curp,{})['nombre']=nombre
        dipus.get(curp,{})['tipoElección']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract_first()
        dipus.get(curp,{})['entidad']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[1]
        parti=response.xpath('//table[@class="cajasombra"]//img/@src').extract()[1]
        dipus.get(curp,{})['partido']=obtienePartido(parti)
        dipus.get(curp,{})['distrito']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[2]
        # dc_dict['?????']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[3]
        dipus.get(curp,{})['correo']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[4]
        dipus.get(curp,{})['extensión']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[5]
        dipus.get(curp,{})['onomástico']=cumple
        supl=response.css('td.textocurri:nth-of-type(2)::text').extract()[6]
        dipus.get(curp,{})['suplente']=limpiaText(supl)
        dipus.get(curp,{})['comisiones']=response.xpath('//a[contains(@href, "integrantes_de_comision")]/text()').getall()
        dipus.get(curp,{})['último grado de estudios']=response.xpath('//td[@class="textoNegro"]/text()').extract_first()
        link_asis=urlDipu + response.css('table.tableInfo  a::attr(href)').extract()[0]
        link_votaciones=urlDipu + response.css('table.tableInfo  a::attr(href)').extract()[1]
        yield response.follow(url=link_asis, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse2)
        yield response.follow(url=link_votaciones, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse4)
    
    def parse2(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse3)
    
    def parse3(self,response):
        curp = response.meta['curp']
        prueba = response.xpath('//tr/td/div/font/text()').extract()
        dias = list(filter(lambda a: (a == 'A' or a =='AC'), prueba))
        dipus.get(curp,{})['Asistencias']=dipus.get(curp,{}).get('Asistencias',0) + len(dias)

    def parse4(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse5)
                
    def parse5(self,response):
        curp = response.meta['curp']
        aux = response.xpath('//td[@class="smallVerde"]/text()').extract()
        aux2={}
        for i in range(1,len(aux),4):
            votos = {}
            votacion = response.xpath('//td[@class="smallVerde"]/text()').extract()[i]
            posi = response.xpath('//td[@class="smallVerde"]/text()').extract()[i+2]
            if(votacion in por_votaciones.keys()):
                por_votaciones[votacion][curp]=posi
            else:
                por_votaciones[votacion] = {curp:posi}
            votos['votación']=votacion
            votos['posición']=posi
            aux2['votos']=aux2.get('votos', []) + [votos]
            #dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos', []) + [votos]
        dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos',0)+len(aux2.get('votos', [])) # Se comenta si la de arriba se descomenta


        

In [3]:
def creaCurp(nombre,cumple):
    
    iniciales = ''.join([c for c in nombre[1:] if c.isupper()])
    c = cumple.replace('-', ' ').split(' ')
    if(len(c[0])==1):
        c[0]="0"+c[0] 
    curp = iniciales[0:3]+c[0]+c[1][0:3]+c[2][-2:]
    return curp

In [4]:
def limpiaText(nom):
    resp = re.sub('\s+',' ',nom)
    resp = resp[0:len(resp)-1]
    return resp

In [5]:
def obtienePartido(imagen):
    res=""
    if(imagen=='images/pan.png'):
        res = "PAN"
    elif(imagen=='images/Logo_PT.png'):
        res = "PT"
    elif(imagen=='images/LogoMorena.jpg'):
        res = "Morena"
    elif(imagen=='images/encuentro.png'):
        res = "Encuentro Social"
    elif(imagen=='images/logo_movimiento_ciudadano.png'):
        res = "Movimiento Ciudadano"
    elif(imagen=='images/pri01.png'):
        res = "PRI"
    elif(imagen=='images/logvrd.jpg'):
        res = "Partido Verde"
    elif(imagen=='images/prd01.png'):
        res = "PRD"
    elif(imagen=='images/logo_SP.jpg'):
         res = "Sin Partido"
    else:
         res = imagen
    return res
                                    

In [6]:
dipus={}
curp = "h"
urlDipu = "http://sitl.diputados.gob.mx/LXIV_leg/"
urlCurr = "http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt="

por_votaciones = {}

terminaciones = []
for i in range(0,5):
    for j in range(0,10):
        for k in range(0,10):
            terminaciones.append(str(i)+str(j)+str(k))
terminaciones.append(str(500))
terminaciones.pop(0)


# Run the Spider
process = CrawlerProcess()
process.crawl(Diputados)
process.start()


2021-03-25 08:43:28 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-25 08:43:28 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Windows-10-10.0.19041-SP0
2021-03-25 08:43:28 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-25 08:43:28 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-25 08:43:33 [scrapy.extensions.telnet] INFO: Telnet Password: ca25c51925e61287
2021-03-25 08:43:36 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-03-25 08:43:45 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddl

2021-03-25 08:43:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=003> (referer: http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=003)
2021-03-25 08:43:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=003> (referer: http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=003)
2021-03-25 08:43:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=018> (referer: http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=018)
2021-03-25 08:43:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=020> (referer: http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=020)
2021-03-25 08:43:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.

2021-03-25 08:43:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=19&pert=16> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=019)
2021-03-25 08:43:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=16&pert=15> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=016)
2021-03-25 08:43:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=16&pert=11> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=016)
2021-03-25 08:43:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=22&pert=18> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=0

2021-03-25 08:44:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=19&pert=4> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=019)
2021-03-25 08:44:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=19&pert=13> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=019)
2021-03-25 08:44:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=19&pert=11> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=019)
2021-03-25 08:44:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=19&pert=9> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?di

2021-03-25 08:44:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=20&pert=3> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=020)
2021-03-25 08:44:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=20&pert=1> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=020)
2021-03-25 08:44:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=18&pert=16> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=018)
2021-03-25 08:44:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=18&pert=15> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?

...

2021-03-25 09:22:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=483&pert=11> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=483)
2021-03-25 09:22:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=483&pert=8> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=483)
2021-03-25 09:22:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=499> (referer: None)
2021-03-25 09:22:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=483&pert=9> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=483)
2021-03-25 09:22:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx

2021-03-25 09:22:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=497&pert=15> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=497)
2021-03-25 09:22:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=497&pert=14> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=497)
2021-03-25 09:22:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=497&pert=11> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=497)
2021-03-25 09:22:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=497&pert=8> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dip

2021-03-25 09:23:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=500&pert=11> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=500)
2021-03-25 09:23:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=500&pert=9> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=500)
2021-03-25 09:23:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=500&pert=8> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=500)
2021-03-25 09:23:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=500&pert=7> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=

2021-03-25 09:23:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=500&pert=1> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=500)
2021-03-25 09:23:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=499&pert=11> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=499)
2021-03-25 09:23:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=499&pert=8> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=499)
2021-03-25 09:23:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=499&pert=7> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dip

2021-03-25 09:23:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=499&pert=4> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=499)
2021-03-25 09:23:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=496&pert=18> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=496)
2021-03-25 09:23:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=499&pert=1> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=499)
2021-03-25 09:23:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=496&pert=16> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?

In [7]:
with open('diputados.json', 'a') as f:
            json.dump(dipus, f, indent=4, ensure_ascii=False)

In [8]:
with open('votaciones.json', 'a') as f:
            json.dump(por_votaciones, f, indent=4, ensure_ascii=False)

In [7]:
for key, value in dipus.items():
    print('\n',key, ' : \n\n', value)


 ELT15mar77  : 

 {'nombre': 'Dip. Elba Lorena Torres Díaz (LICENCIA)', 'tipoElección': 'Mayoría Relativa', 'entidad': 'Aguascalientes ', 'partido': 'PT', 'distrito': '2 ', 'correo': 'elba.torres@diputados.gob.mx ', 'extensión': '62100 ', 'onomástico': '15-marzo-1977 ', 'suplente': 'Haydeé Hernández Muñoz', 'comisiones': [' Derechos Humanos  ', ' Salud (Secretaría) '], 'último grado de estudios': 'Licenciatura', 'Asistencias': 130, 'votos': 610}

 FJL01jul79  : 

 {'nombre': 'Dip. Francisco Javier Luévano Núñez', 'tipoElección': 'Mayoría Relativa', 'entidad': 'Aguascalientes ', 'partido': 'PAN', 'distrito': '1 ', 'correo': 'francisco.luevano@diputados.gob.mx ', 'extensión': '59907 ', 'onomástico': '1-julio-1979 ', 'suplente': 'Gustavo Armendariz Viramontes', 'comisiones': [' Asuntos Migratorios  ', ' Comunicaciones y Transportes (Secretaría) ', ' Presupuesto y Cuenta Pública  ', ' Alemania (Vicepresidencia) ', ' Bielorrusia  ', ' Israel  ', ' Japón (Vicepresidencia) ', ' Paraguay  ', 

In [7]:
for key, value in por_votaciones.items():
    print('\n',key,': ')
    for curp, posi in value.items():
        print('\n',curp, ': ', posi)


 DECRETO POR EL QUE SE ADICIONA UN PÁRRAFO AL ARTÍCULO 1° DE LA LEY DE ADQUISICIONES ARRENDAMIENTOS Y SERVICIOS DEL SECTOR PÚBLICO (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE TRANSPARENCIA Y ACCESO A LA INFORMACIÓN PÚBLICA, DE LA LEY GENERAL DEL SISTEMA NACIONAL ANTICORRUPCIÓN, DE LA LEY FEDERAL DE COMPETENCIA ECONÓMICA, DE LA LEY FEDERAL DE TELECOMUNICACIONES Y RADIODIFUSIÓN, DE LA LEY DEL BANCO DE MÉXICO, DE LA LEY DE LA COMISIÓN NACIONAL DE LOS DERECHOS HUMANOS, DE LA LEY ORGÁNICA DE LA FISCALÍA GENERAL DE LA REPÚBLICA Y DE LA LEY DE LOS ÓRGANOS REGULADORES COORDINADOS EN MATERIA ENERGÉTICA, EN MATERIA DE PARIDAD DE GÉNERO (EN LO GNERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY FEDERAL DE DEFENSORÍA PÚBLICA, DE LA LEY GENERAL DE SALUD, DE LA LEY 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE EXPIDE LA LEY GENERAL DE EDUCACIÓN SUPERIOR Y SE ABROGA LA LEY PARA LA COORDINACIÓN DE LA EDUCACIÓN SUPERIOR (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE EXPIDE LA LEY GENERAL DE EDUCACIÓN SUPERIOR Y SE ABROGA LA LEY PARA LA COORDINACIÓN DE LA EDUCACIÓN SUPERIOR (EN LO PARTICULAR LOS ARTÍCULOS RESERVADOS, EN TÉRMINOS DEL DICTAMEN)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  Abstención

 DECRETO POR EL QUE SE EXPIDE LA LEY FEDERAL PARA LA REGULACIÓN DEL CANNABIS, Y SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE SALUD Y DEL CÓDIGO PENAL FEDERAL (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE EXPIDE LA LEY FEDERAL PARA LA REGULACIÓN DEL CANNABIS, Y SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE S

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 4 DE LA LEY DEL INSTITUTO NACIONAL DE LOS PUEBLOS INDÍGENAS (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMA LA FRACCIÓN III DEL ARTÍCULO 75 DE LA LEY GENERAL DEL SISTEMA NACIONAL DE SEGURIDAD PÚBLICA (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN Y ADICIONAN LOS ARTÍCULOS 261 Y 366 DEL CÓDIGO NACIONAL DE PROCEDIMIENTOS PENALES (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN LOS ARTÍCULOS 423 DEL CÓDIGO CIVIL FEDERAL, 170 DE LA LEY GENERAL DE EDUCACIÓN Y 47 DE LA LEY GENERAL DE LOS DERECHOS DE NIÑAS, NIÑOS Y ADOLESCENTES (EN LO GENERAL Y EN LO PARTICULAR CON LAS MODIFICACIONES ACEPTADAS)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN, ADICIONAN Y DEROGAN DIVERSAS DISPOSIC

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE ADICIONA UN ARTICULO 254 QUÁTER AL CÓDIGO PENAL FEDERAL (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE ADICIONA UN ARTÍCULO 258 BIS AL CÓDIGO NACIONAL DE PROCEDIMIENTOS PENALES (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE VÍCTIMAS (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE VÍCTIMAS (EN LO PARTICULAR LOS ARTÍCULOS RESERVADOS EN TÉRMINOS DEL DICTAMEN Y CON LAS MODIFICACIONES ACEPTADAS POR LA ASAMBLEA )  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE ADICIONAN LOS ARTÍCULOS 95 DE LA LEY ORGÁNICA DEL PODER JUDICIAL DE LA FEDERACIÓN Y 36 DE LA 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE EL HONORABLE CONGRESO DE LA UNIÓN, DECLARA EL 15 DE SEPTIEMBRE DE CADA AÑO, COMO EL "DÍA NACIONAL DE LA LUCHA CONTRA EL LINFOMA NO HODGKING Y LINFOMA HODGKING" (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  Abstención

 DECRETO POR EL QUE EL HONORABLE CONGRESO DE LA UNIÓN, DECLARA EL 9 DE AGOSTO DE CADA AÑO, COMO EL "DÍA NACIONAL DE LA LUCHA CONTRA EL CÁNCER CERVICOUTERINO" (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN Y ADICIONAN LOS ARTÍCULOS 88 BIS Y 120 DE LA LEY DE AGUAS NACIONALES (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN LOS ARTÍCULOS 62 Y 63 DEL CÓDIGO CIVIL FEDERAL (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMA LA FRACCIÓN VI DEL ARTÍCULO 112 DE LA LEY GENERA

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN, ADICIONAN Y DEROGAN DIVERSAS DISPOSICIONES DE LA LEY DE PREMIOS, ESTÍMULOS Y RECOMPENSAS CIVILES (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DE DECRETO POR EL QUE SE REFORMA EL PRIMER PÁRRAFO DEL ARTÍCULO 13 DE LA LEY DEL SERVICIO DE ADMINISTRACIÓN TRIBUTARIA (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMA LA DENOMINACIÓN DEL CAPÍTULO IV Y EL ARTÍCULO 189 DEL CÓDIGO PENAL FEDERAL (EN LO GENERAL Y EN LO PARTICULAR CON LA MODIFICACIÓN ACEPTADA POR LA ASAMBLEA)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO QUE REFORMA EL PRIMER PÁRRAFO DEL ARTÍCULO 390 DEL CÓDIGO PENAL FEDERAL (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 187 Y ADICIONAL EL ARTÍCULO 192 DEL CÓDIGO NACIONAL DE PROCEDIMIENTOS PENALES (EN LO GENERAL Y EN L

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE EXPIDE LA LEY DE AUSTERIDAD REPUBLICANA; Y SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE RESPONSABILIDADES ADMINISTRATIVAS Y DE LA LEY FEDERAL DE PRESUPUESTO Y RESPONSABILIDAD HACENDARIA (EN LO PARTICULAR LOS ARTÍCULOS 17 Y 24 RESERVADOS, EN TÉRMINOS DEL DICTAMEN)  : 

 MIA24feb66 :  En contra

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN LOS ARTÍCULOS 116 Y 127 DE LA CONSTITUCIÓN POLÍTICA DE LOS ESTADOS UNIDOS MEXICANOS, EN MATERIA DE REMUNERACIÓN DE SERVIDORES PÚBLICOS (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN LOS ARTÍCULOS 116 Y 127 DE LA CONSTITUCIÓN POLÍTICA DE LOS ESTADOS UNIDOS MEXICANOS, EN MATERIA DE REMUNERACIÓN DE SERVIDORES PÚBLICOS (EN LO PARTICULAR LOS ARTÍCULOS 116 y 127 RESERVADOS, EN TÉRMINOS DEL DICTAMEN)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DEC


 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN LA DENOMINACIÓN DEL CAPÍTULO TERCERO, LOS ARTÍCULOS 17, FRACCIÓN III, 35 Y 36 DE LA LEY GENERAL PARA LA IGUALDAD ENTRE MUJERES Y HOMBRES (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMA LA FRACCIÓN III DEL ARTÍCULO 994 DE LA LEY FEDERAL DEL TRABAJO (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE ADICIONA EL ARTÍCULO 20 BIS A LA LEY GENERAL EN MATERIA DE DELITOS ELECTORALES (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE ACCESO DE LAS MUJERES A UNA VIDA LIBRE DE VIOLENCIA Y DE LA LEY GENERAL DE SALUD, EN MATERIA DE VIOLENCIA OBSTÉTRICA (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MIA24feb66 :  A favor

 SOB20ago83 :  A favor

 DECRETO POR EL QUE SE REFORM

In [18]:
limpiaText("Ma. Guadalupe Loredo Gonzáles\n\t\t\t\t\t\n\n\n ")

'Ma. Guadalupe Loredo Gonzáles'